# Sentiment Analysis and Opinion Mining

https://learn.microsoft.com/en-us/azure/ai-services/language-service/sentiment-opinion-mining/overview?wt.mc_id=MVP_322781

## Install Library

In [ ]:
%pip install azure-ai-textanalytics

## Load Azure Configurations

In [1]:
import os

# Load Azure configurations from environment variables
# Ensure that AZURE_AI_LANGUAGE_KEY and AZURE_AI_LANGUAGE_ENDPOINT are set in your environment
language_key = os.environ.get('AZURE_AI_LANGUAGE_KEY')
language_endpoint = os.environ.get('AZURE_AI_LANGUAGE_ENDPOINT')

## Create a Text Analytics client

In [2]:
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using Azure Key and Endpoint
def authenticate_client():
    """
    Authenticates the Azure Text Analytics client using the provided key and endpoint.

    Returns:
        TextAnalyticsClient: An authenticated client for Azure Text Analytics.
    """
    ta_credential = AzureKeyCredential(language_key)
    text_analytics_client = TextAnalyticsClient(
        endpoint=language_endpoint,
        credential=ta_credential
    )
    return text_analytics_client

# Initialize the client
client = authenticate_client()

## Analyze Sentiment function

In [3]:
def sentiment_analysis_with_opinion_mining(client, documents):
    """
    Analyzes the sentiment of the provided documents and extracts opinions using the Azure Text Analytics client.

    Args:
        client (TextAnalyticsClient): An authenticated Azure Text Analytics client.
        documents (list of str): A list of text documents to analyze for sentiment and opinions.

    Returns:
        None: Prints the sentiment analysis results, including overall sentiment, sentence-level sentiment, 
              and mined opinions for each document.
    """
    # Call the Azure Text Analytics API to analyze sentiment with opinion mining
    result = client.analyze_sentiment(documents, show_opinion_mining=True)

    # Filter out documents with errors
    doc_result = [doc for doc in result if not doc.is_error]

    # Iterate through the results for each valid document
    for doc_idx, document in enumerate(doc_result, start=1):  # Start index at 1 for readability
        # Print the overall sentiment of the document
        print("Document {} Sentiment: {}".format(doc_idx, document.sentiment))
        print("Overall scores: positive={0:.2f}; neutral={1:.2f}; negative={2:.2f}".format(
            document.confidence_scores.positive,
            document.confidence_scores.neutral,
            document.confidence_scores.negative,
        ))

        # Iterate through each sentence in the document
        for idx, sentence in enumerate(document.sentences, start=1):
            # Print the sentiment of the sentence
            print("  Sentence {}: {}".format(idx, sentence.text))
            print("    Sentiment: {}".format(sentence.sentiment))
            print("    Scores: Positive={0:.2f}, Neutral={1:.2f}, Negative={2:.2f}".format(
                sentence.confidence_scores.positive,
                sentence.confidence_scores.neutral,
                sentence.confidence_scores.negative,
            ))

            # Initialize a counter for mined opinions
            opinion_counter = 1

            # Iterate through mined opinions in the sentence
            for mined_opinion in sentence.mined_opinions:
                # Label the mined opinion
                print(f"    Mined Opinion {opinion_counter}:")
                opinion_counter += 1

                # Print the target of the opinion
                target = mined_opinion.target
                print("      Target '{}' ({})".format(target.text, target.sentiment))
                print("      Scores: Positive={0:.2f}, Negative={1:.2f}".format(
                    target.confidence_scores.positive,
                    target.confidence_scores.negative,
                ))

                # Print the assessments related to the target
                for assessment in mined_opinion.assessments:
                    print("        Assessment '{}' ({})".format(assessment.text, assessment.sentiment))
                    print("        Scores: Positive={0:.2f}, Negative={1:.2f}".format(
                        assessment.confidence_scores.positive,
                        assessment.confidence_scores.negative,
                    ))

        # Add a blank line after each document for better readability
        print()

In [4]:
documents = [
    """I had the best day of my life. 
    I decided to go sky-diving and it made me appreciate my whole life so much more.
    I developed a deep-connection with my instructor as well, and I feel as if I've made a life-long friend in her.""",

    """This was a waste of my time. 
    All of the views on this drop are extremely boring, all I saw was grass. 
    0/10 would not recommend to any divers, even first timers.""",
    
    """The staff was incredibly friendly, the rooms were spotless, and the location was perfect for sightseeing. 
    Additionally, the breakfast buffet was delicious, the pool area was well-maintained, and the concierge service was very helpful."""
]

sentiment_analysis_with_opinion_mining(client, documents)

Document 1 Sentiment: positive
Overall scores: positive=0.97; neutral=0.03; negative=0.00
  Sentence 1: I had the best day of my life.      
    Sentiment: positive
    Scores: Positive=0.98, Neutral=0.02, Negative=0.00
  Sentence 2: I decided to go sky-diving and it made me appreciate my whole life so much more.     
    Sentiment: positive
    Scores: Positive=0.96, Neutral=0.04, Negative=0.00
    Mined Opinion 1:
      Target 'sky-diving' (positive)
      Scores: Positive=1.00, Negative=0.00
        Assessment 'appreciate' (positive)
        Scores: Positive=1.00, Negative=0.00
  Sentence 3: I developed a deep-connection with my instructor as well, and I feel as if I've made a life-long friend in her.
    Sentiment: neutral
    Scores: Positive=0.26, Neutral=0.73, Negative=0.01

Document 2 Sentiment: negative
Overall scores: positive=0.00; neutral=0.00; negative=1.00
  Sentence 1: This was a waste of my time.      
    Sentiment: negative
    Scores: Positive=0.00, Neutral=0.00, Neg